<a href="https://colab.research.google.com/github/Risskr/Stock-App/blob/Production_v2/Solar_System_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set Up

In [ ]:
#Set up

#Read CSVs

In [ ]:
import pandas as pd
# Import Nasdaq_df CSV
#SAVE_PATH_NASDAQ = '/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv'
#nasdaq_df = pd.read_csv({SAVE_PATH_NASDAQ})
#latest_date_nasdaq_data = nasdaq_df['date'].max().strftime('%Y%m%d')

# Import Three Month Correlation CSV
three_month_file = f'/content/drive/MyDrive/Colab Notebooks/Production/three_month_spearman_lagged_correlation.csv'
three_month_spearman_lagged_correlations = pd.read_csv(three_month_file, index_col=0)

# Import Six Month Correlation CSV
six_month_file = f'/content/drive/MyDrive/Colab Notebooks/Production/six_month_spearman_lagged_correlation.csv'
six_month_spearman_lagged_correlations = pd.read_csv(six_month_file, index_col=0)

# Import Screener_data
SAVE_PATH_SCREENER = '/content/drive/MyDrive/Colab Notebooks/Production/screener_data_df.csv'
screener_data_df = pd.read_csv(SAVE_PATH_SCREENER)

# Import Top Predictions
SAVE_PATH_TOP_PREDICTIONS = '/content/drive/MyDrive/Colab Notebooks/Production/top_gravitational_impacts.csv'
top_predictions_df = pd.read_csv(SAVE_PATH_TOP_PREDICTIONS)

#Solar System Parameters
min_nodes = 5
max_nodes = 30
threshold_percent = 0.9

# Default source Ticker
source_ticker = 'AAPL'



## Process correlated Data and get gravitational scores

In [ ]:
import pandas as pd
import numpy as np

# Function to process stock correlation data, calculate gravitational forces,
# and filter connections for visualization based on the force.

def process_and_score_stocks(
    six_month_correlations,
    three_month_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent
):
    """
    Processes stock correlation data for a specific source ticker.
    It filters for positive correlations, computes a dynamic impact score (gravitational_force),
    filters connections, and then calculates a final net gravitational force and the
    maximum potential force under ideal conditions.

    Args:
      six_month_correlations: The six-month spearman lagged correlation matrix.
      three_month_correlations: The three-month spearman lagged correlation matrix.
      screener_data_df: DataFrame with additional stock information.
      source_ticker: The ticker symbol for which to process data.
      min_nodes: Minimum number of correlated stocks to return.
      max_nodes: Maximum number of correlated stocks to return.
      threshold_percent: A percentage (0.0 to 1.0) of the maximum force to use as a filter.

    Returns:
      processed_data_df: A pandas DataFrame with processed data for visualization.
      source_data_df: A pandas DataFrame containing the net_gravitational_force,
                      max_potential_force, and gravitational_impact for the source ticker,
                      along with the source ticker's market cap influence and source_planet_radius.
    """
    # --- Data Unpivoting and Initial Setup ---
    # Start with the 6-month correlation data as the base
    correlation_df = six_month_correlations.rename_axis('source', axis=0)
    grouped_correlation_data = correlation_df.stack().reset_index()
    grouped_correlation_data.columns = ['source', 'target', 'six_month_spearman_correlation']

    grouped_correlation_data = grouped_correlation_data[
        (grouped_correlation_data['source'] != grouped_correlation_data['target']) &
        (grouped_correlation_data['target'] != source_ticker)
    ].copy()

    # --- Filter for the specific source ticker ---
    source_connections = grouped_correlation_data[grouped_correlation_data['source'] == source_ticker].copy()
    if source_connections.empty:
        print(f"No correlation data found for source ticker {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # Add 3-month correlation data before filtering
    source_connections['three_month_spearman_correlation'] = source_connections.apply(
        lambda row: three_month_correlations.loc[row['source'], row['target']] if row['source'] in three_month_correlations.index and row['target'] in three_month_correlations.columns else 0, axis=1
    )

    # We only care about positively correlated stocks for this model in both 6 and 3 month periods
    positive_corr_group = source_connections[
        (source_connections['six_month_spearman_correlation'] > 0) &
        (source_connections['three_month_spearman_correlation'] > 0)
    ].copy()

    if positive_corr_group.empty:
        print(f"No positive correlations found for source ticker {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # --- Enrich Data (before filtering) ---
    # Add market data
    screener_cols_to_add = ['code', 'market_capitalization', 'last_day_change']
    required_screener_cols = ['code', 'market_capitalization', 'last_day_change']
    if not all(col in screener_data_df.columns for col in required_screener_cols):
        missing = [col for col in required_screener_cols if col not in screener_data_df.columns]
        raise ValueError(f"screener_data_df is missing required columns: {missing}")

    screener_info = screener_data_df[screener_cols_to_add].rename(columns={'code': 'target'})
    positive_corr_group = pd.merge(positive_corr_group, screener_info, on='target', how='left')
    positive_corr_group.dropna(subset=['market_capitalization', 'last_day_change'], inplace=True)
    if positive_corr_group.empty:
        print(f"No valid connections after merging screener data for {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()


    # --- Calculate Dynamic Impact Score (Gravitational Force) ---
    epsilon = 1e-9 # Small value to avoid log(0) issues.
    # Weights for recency bias
    w_3m = 0.6
    w_6m = 0.4
    # "unified_correlation" is a weighted average of recent correlations.
    positive_corr_group['unified_correlation'] = (
        w_3m * positive_corr_group['three_month_spearman_correlation'] +
        w_6m * positive_corr_group['six_month_spearman_correlation']
    )

    # Calculate a market cap influence score scaled between 0 and 1 for target stocks.
    positive_corr_group['Market Cap'] = positive_corr_group['market_capitalization']

    # --- Calculate source ticker's market cap and log cap ---
    source_screener_info = screener_data_df[screener_data_df['code'] == source_ticker]
    source_market_cap = source_screener_info['market_capitalization'].iloc[0] if not source_screener_info.empty and 'market_capitalization' in source_screener_info.columns else epsilon
    source_log_cap = np.log(max(source_market_cap, epsilon))


    # Calculate log market caps for all relevant tickers (source and targets)
    all_market_caps = positive_corr_group['Market Cap'].tolist()
    all_market_caps.append(source_market_cap) # Include source market cap

    log_caps = np.log(pd.Series(all_market_caps).clip(lower=epsilon))

    min_log_cap, max_log_cap = log_caps.min(), log_caps.max()
    log_cap_range = max_log_cap - min_log_cap

    # Calculate market cap influence for target stocks
    if log_cap_range > 0:
        positive_corr_group['market_cap_influence'] = np.log(positive_corr_group['Market Cap'].clip(lower=epsilon))
    else:
        positive_corr_group['market_cap_influence'] = 20 # Neutral value if all caps are the same


    # The `gravitational_force` is a product of recent correlation strength and market influence.
    # Modified: Increased the influence of unified_correlation by multiplying by a factor
    correlation_weight_factor = 1.0 # Factor to increase the influence of unified_correlation
    positive_corr_group['gravitational_force'] = (
        (positive_corr_group['unified_correlation'] * correlation_weight_factor) * # Multiply unified_correlation by a factor
        positive_corr_group['market_cap_influence']
    )

    # --- Apply Filtering ---
    max_abs_force = positive_corr_group['gravitational_force'].abs().max()
    if pd.isna(max_abs_force) or max_abs_force == 0:
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    force_threshold = max_abs_force * threshold_percent
    filtered_by_force_threshold = positive_corr_group[positive_corr_group['gravitational_force'].abs() >= force_threshold].copy()

    # Enforce min/max node constraints
    if len(filtered_by_force_threshold) < min_nodes:
        final_filtered_df = positive_corr_group.sort_values(by='gravitational_force', key=abs, ascending=False).head(min_nodes).copy()
    elif len(filtered_by_force_threshold) > max_nodes:
        final_filtered_df = filtered_by_force_threshold.sort_values(by='gravitational_force', key=abs, ascending=False).head(max_nodes).copy()
    else:
        final_filtered_df = filtered_by_force_threshold.copy()

    if final_filtered_df.empty:
        print(f"No connections remained for {source_ticker} after filtering.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # --- Calculate Final Net Force and Visualization Parameters ---
    final_filtered_df['Daily Change'] = final_filtered_df['last_day_change']

    final_filtered_df['signed_gravitational_force'] = final_filtered_df.apply(
        lambda row: row['gravitational_force'] if row['Daily Change'] >= 0 else -row['gravitational_force'],
        axis=1
    )

    net_gravitational_force = final_filtered_df['signed_gravitational_force'].sum()
    max_potential_force = final_filtered_df['market_cap_influence'].sum()

    # --- Calculate Visualization Parameters ---
    min_corr, max_corr = final_filtered_df['gravitational_force'].min(), final_filtered_df['gravitational_force'].max()
    corr_range = max_corr - min_corr if max_corr > min_corr else 1.0
    # MODIFIED: Reverse the scaling for Orbital Radius
    if corr_range > 0:
        final_filtered_df['Orbital Radius'] = 1 - ((final_filtered_df['gravitational_force'] - min_corr) / corr_range)
    else:
        final_filtered_df['Orbital Radius'] = 0.5 # Neutral value if all forces are the same

    # -----Calculate Planet Radius------
    # Combine all market caps to find the true min and max for normalization
    all_caps = pd.concat([
        final_filtered_df['Market Cap'],
        pd.Series([source_market_cap]) # Make sure source_market_cap is a Series
    ], ignore_index=True)

    # Calculate the log, clipping to avoid errors with zero
    log_all_caps = np.log(all_caps.clip(lower=epsilon))

    # Find the min and max from the complete set of data
    min_log_cap = log_all_caps.min()
    max_log_cap = log_all_caps.max()
    log_cap_range = max_log_cap - min_log_cap

    # Now, apply the normalization ONLY to the DataFrame's data
    # using the min/max from the combined set
    if log_cap_range > 0:
        # We are calculating log on just the dataframe column now
        log_df_caps = np.log(final_filtered_df['Market Cap'].clip(lower=epsilon))
        final_filtered_df['Planet Radius'] = (log_df_caps - min_log_cap) / log_cap_range
    else:
        # If all values are the same, assign a default radius
        final_filtered_df['Planet Radius'] = 0.5

    # Calculate source_planet_radius using the same min/max log caps from the targets and source.
    if log_cap_range > 0:
        source_planet_radius = (source_log_cap - min_log_cap) / log_cap_range
    else:
        source_planet_radius = 0.5 # Neutral value if all caps are the same

    # --- Final Cleanup and Column Selection ---
    # "gravitational_percent" shows the relative % contribution of each stock.
    final_filtered_df['gravitational_percent'] = (final_filtered_df['signed_gravitational_force'] / final_filtered_df['gravitational_force'].sum()) * 100

    final_columns = [
        'source', 'target', 'Daily Change', 'six_month_spearman_correlation',
        'three_month_spearman_correlation', 'unified_correlation',
        'Orbital Radius', 'Market Cap', 'Planet Radius', 'market_cap_influence',
        'gravitational_force', 'signed_gravitational_force', 'gravitational_percent'
    ]


    gravitational_impact = (net_gravitational_force / max_potential_force) * 100 if max_potential_force > 0 else 0

    # Use the same min_log_cap and log_cap_range from target stocks for scaling
    source_market_cap_influence = 20 if log_cap_range <= 0 else (source_log_cap)

    # Create source_data_df
    source_data_df = pd.DataFrame([{
        'ticker': source_ticker,
        'net_gravitational_force': net_gravitational_force,
        'max_potential_force': max_potential_force,
        'gravitational_impact': gravitational_impact,
        'source_market_cap_influence': source_market_cap_influence, # Add the source influence
        'source_planet_radius': source_planet_radius # Add the source planet radius
    }])


    for col in final_columns:
        if col not in final_filtered_df.columns:
            final_filtered_df[col] = np.nan

    processed_data_df = final_filtered_df[final_columns].copy()

    return processed_data_df, source_data_df


# Process the data for the network diagram
processed_data_df, source_data_df = process_and_score_stocks(
    six_month_spearman_lagged_correlations,
    three_month_spearman_lagged_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent,
    )

# Extract the scalar values from the source_data_df for plotting
net_gravitational_force = source_data_df['net_gravitational_force'].iloc[0]
max_potential_force = source_data_df['max_potential_force'].iloc[0]
gravitational_impact = source_data_df['gravitational_impact'].iloc[0]
market_cap_influence = source_data_df['source_market_cap_influence'].iloc[0]
source_planet_radius = source_data_df['source_planet_radius'].iloc[0]

# print(f"Net Gravitational Force: {net_gravitational_force:.2f}")
# print(f"Max Potential Gravitational Force: {max_potential_force:.2f}")
# print(f"Net Gravitaional Impact: {gravitational_impact:.2f}%")
# print(f"Source Market Cap Influence: {market_cap_influence}")
# print(f"Source Planet Radius: {source_planet_radius}")
# print('----------------------------------')
# processed_data_df

##List of Top Predictions

In [ ]:
# Filter Top Predictions for visual aid

# Sort by gravitational_impact in descending order for top positive impacts
top_positive_impacts = gravitational_impact_df.sort_values(by='gravitational_impact', ascending=False).head(10).reset_index(drop=True)

# Sort by gravitational_impact in ascending order for top negative impacts
top_negative_impacts = gravitational_impact_df.sort_values(by='gravitational_impact', ascending=True).head(10).reset_index(drop=True)


#Setup Ngrok

In [ ]:
# Ngrok Setup

#Terminal Command
#ngrok config add-authtoken 2yPWVwCXJDsY9JkhLhnrn549R5E_5b2uFuh7R4ER3fLJWLU59

# Install necessary libraries (run this cell first)
!pip install -q pyngrok

# --- Import necessary libraries ---
import os
from pyngrok import ngrok
from google.colab import userdata

# --- Configure Ngrok for public access ---
# This part remains the same to create a public URL for your app
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    # Use pyngrok to set the auth token programmatically
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
except userdata.SecretNotFoundError:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")
    # You might want to handle this case, e.g., by exiting or using a default token if applicable
    exit()


# Define the port your Dash app will run on.
DASH_PORT = 8051

In [ ]:
# Create a public ngrok tunnel to the specified port.
print("Establishing Ngrok tunnel...")
public_url = ngrok.connect(DASH_PORT)
print(f"🚀 Your Dash app will be accessible at: {public_url}")

In [ ]:
# Kills all ngrok processes that pyngrok started
print("Attempting to kill all active ngrok tunnels...")
ngrok.kill()
print("All ngrok tunnels should now be terminated.")

# Add a short delay to ensure the process is killed
import time
time.sleep(3)

# Dash App

In [ ]:
# Dash App Setup

# Install necessary libraries (run this cell first)
!pip install -q dash dash-bootstrap-components plotly

# --- Import necessary libraries ---
from dash import Dash, html, dcc, Input, Output # We need dcc for interactive components and Input/Output for callbacks


In [ ]:
# This is a dash app by Chris Parks
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from dash import Dash, dcc, html, callback_context
from dash.dependencies import Input, Output, State, ALL
from dash.exceptions import PreventUpdate
import json
import base64
from datetime import datetime

# --- Helper Functions ---

def create_model_image_svg(base_color, subdivisions, texture_spots_count):
    """
    Creates a base64 encoded SVG data URL that is a 2D representation of the 3D low-poly model.
    This function generates the geometry, projects it, and simulates flat shading to match the plot.
    """
    # 1. Generate the 3D model's vertices and faces for a unit sphere
    # We pass 0 for texture_spots here because we'll draw them separately in the SVG
    vertices, faces, _ = create_low_poly_sphere(0, 0, 0, 1, base_color, subdivisions, 0)

    # 2. Define a light source for shading the facets
    light_source = np.array([-0.5, 0.8, 1.0])
    light_source = light_source / np.linalg.norm(light_source)

    # 3. Process each face for rendering
    face_data = []
    for face in faces:
        # Get the vertices for the current face
        v0, v1, v2 = vertices[face]

        # --- Back-face culling: Don't render faces pointing away from the camera ---
        # The camera is at (0, 0, z), so we check the z-component of the normal
        normal = np.cross(v1 - v0, v2 - v0)
        if np.linalg.norm(normal) == 0: continue
        normal = normal / np.linalg.norm(normal)
        if normal[2] < 0:
            continue # This face is on the back of the sphere, so we skip it

        # --- Shading: Calculate brightness based on angle to the light source ---
        intensity = np.dot(normal, light_source)
        # Map intensity to a brightness factor for the color
        color_factor = 0.65 + intensity * 0.5
        facet_color = darken_color(base_color, color_factor)

        # --- Projection: Convert 3D vertex coordinates to 2D SVG coordinates ---
        # We scale and shift the (x, y) coordinates to fit in a 100x100 SVG
        points_2d_str = " ".join([f"{(v[0] * 48) + 50},{(v[1] * -48) + 50}" for v in [v0, v1, v2]])

        # Store the face's z-depth for sorting, so closer faces draw on top
        avg_z = (v0[2] + v1[2] + v2[2]) / 3
        face_data.append({'z': avg_z, 'points': points_2d_str, 'color': facet_color})

    # 4. Sort faces from back to front
    face_data.sort(key=lambda f: f['z'])

    # 5. Build the SVG polygons from the sorted face data
    svg_polygons = "".join(f'<polygon points="{f["points"]}" fill="{f["color"]}" />' for f in face_data)

    # 6. Add texture spots as random circles
    texture_color = darken_color(base_color, 0.7)
    svg_texture_spots = ""
    np.random.seed(sum(ord(c) for c in base_color)) # Seed for consistency
    for _ in range(texture_spots_count):
        angle = np.random.uniform(0, 2 * np.pi)
        radius = np.random.uniform(0, 48)
        spot_size = np.random.uniform(4, 9)
        cx = 50 + radius * np.cos(angle)
        cy = 50 + radius * np.sin(angle)
        svg_texture_spots += f'<circle cx="{cx}" cy="{cy}" r="{spot_size}" fill="{texture_color}" opacity="0.7"/>'

    # 7. Assemble the final SVG string
    svg_string = f'''
    <svg width="100" height="100" viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg">
      <defs>
        <clipPath id="sphereClip">
          <circle cx="50" cy="50" r="48"/>
        </clipPath>
        <filter id="blur-effect">
          <feGaussianBlur in="SourceGraphic" stdDeviation="0.7" />
        </filter>
      </defs>
      <g clip-path="url(#sphereClip)" filter="url(#blur-effect)">
        {svg_polygons}
        {svg_texture_spots}
      </g>
      <circle cx="50" cy="50" r="48" fill="none" stroke="rgba(255, 255, 255, 0.25)" stroke-width="1.5" />
    </svg>
    '''
    encoded_svg = base64.b64encode(svg_string.encode('utf-8')).decode('utf-8')
    return f"data:image/svg+xml;base64,{encoded_svg}"

def darken_color(color_hex, factor=0.8):
    """Darkens or lightens a hex color by a given factor."""
    rgb = mcolors.to_rgb(color_hex)
    # Clamp values to ensure they stay within the valid [0, 1] range for RGB
    modified_rgb = [min(max(c * factor, 0), 1) for c in rgb]
    return mcolors.to_hex(modified_rgb)

def create_low_poly_sphere(center_x, center_y, center_z, radius, base_color, subdivisions=2, texture_spots=15):
    """
    Generates vertex and face data for a textured low-poly sphere (icosphere).
    """
    # Define the 12 vertices of a regular icosahedron
    t = (1.0 + np.sqrt(5.0)) / 2.0
    vertices = np.array([
        [-1,  t,  0], [ 1,  t,  0], [-1, -t,  0], [ 1, -t,  0],
        [ 0, -1,  t], [ 0,  1,  t], [ 0, -1, -t], [ 0,  1, -t],
        [ t,  0, -1], [ t,  0,  1], [-t,  0, -1], [-t,  0,  1]
    ])

    # Define the 20 triangular faces of the icosahedron
    faces = np.array([
        [0, 11, 5], [0, 5, 1], [0, 1, 7], [0, 7, 10], [0, 10, 11],
        [1, 5, 9], [5, 11, 4], [11, 10, 2], [10, 7, 6], [7, 1, 8],
        [3, 9, 4], [3, 4, 2], [3, 2, 6], [3, 6, 8], [3, 8, 9],
        [4, 9, 5], [2, 4, 11], [6, 2, 10], [8, 6, 7], [9, 8, 1]
    ])

    # Subdivide faces to create more polygons
    for _ in range(subdivisions):
        new_faces = []
        mid_points = {}
        for face in faces:
            v_indices = [face[0], face[1], face[2]]
            new_v_indices = []
            for i in range(3):
                v1 = v_indices[i]
                v2 = v_indices[(i + 1) % 3]
                mid_key = tuple(sorted((v1, v2)))
                mid_idx = mid_points.get(mid_key)
                if mid_idx is None:
                    mid_idx = len(vertices)
                    vertices = np.vstack([vertices, (vertices[v1] + vertices[v2]) / 2.0])
                    mid_points[mid_key] = mid_idx
                new_v_indices.append(mid_idx)
            new_faces.append([v_indices[0], new_v_indices[0], new_v_indices[2]])
            new_faces.append([v_indices[1], new_v_indices[1], new_v_indices[0]])
            new_faces.append([v_indices[2], new_v_indices[2], new_v_indices[1]])
            new_faces.append(new_v_indices)
        faces = np.array(new_faces)

    # Normalize vertices to form a sphere, then scale and translate
    vertices = vertices / np.linalg.norm(vertices, axis=1)[:, np.newaxis]
    final_vertices = vertices * radius + np.array([center_x, center_y, center_z])

    # Create vertex colors for texture spots
    darker_color_hex = darken_color(base_color, 0.7)
    vertex_colors = [base_color] * len(final_vertices)
    if texture_spots > 0:
        spot_indices = np.random.choice(len(final_vertices), size=texture_spots, replace=False)
        for idx in spot_indices:
            vertex_colors[idx] = darker_color_hex

    return final_vertices, faces, vertex_colors

# --- Define New Color Palette and Colormap ---
# Define the red and green color spectrums
RED_SPECTRUM = {'light': '#ee868f', 'dark': '#cd2d35'}
GREEN_SPECTRUM = {'light': '#8ac08b', 'dark': '#329542'}

# Create separate colormaps for negative (red) and positive (green) values
red_cmap = mcolors.LinearSegmentedColormap.from_list('red_cmap', [RED_SPECTRUM['light'], RED_SPECTRUM['dark']])
green_cmap = mcolors.LinearSegmentedColormap.from_list('green_cmap', [GREEN_SPECTRUM['light'], GREEN_SPECTRUM['dark']])

def get_node_color(value, min_val, max_val):
    """
    Determines the node color based on its value, using a diverging red/green scale.
    """
    if value >= 0:
        if value >= max_val:
            return GREEN_SPECTRUM['dark']
        norm_val = value / max_val if max_val != 0 else 0
        return mcolors.to_hex(green_cmap(norm_val))
    else: # value < 0
        if value <= min_val:
            return RED_SPECTRUM['dark']
        norm_val = value / min_val if min_val != 0 else 0
        return mcolors.to_hex(red_cmap(norm_val))


# --- Main Plotting Function ---
def solar_system_visual(source_ticker, processed_data_df, source_data_df, screener_data_df, zoom=1.5):
    """
    Creates the main 3D solar system visualization with a low-poly aesthetic.
    """
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()
    source_info_row = source_data_df[source_data_df['ticker'] == source_ticker]

    if ticker_connections.empty or source_info_row.empty:
        return go.Figure().update_layout(title=f"Data not available for {source_ticker}", title_x=0.5, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')

    source_info = source_info_row.iloc[0]
    fig = go.Figure()

    pos = {source_ticker: (0, 0, 0)}
    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker].copy()
    num_connections = len(actual_target_connections)
    radii_for_rings = []

    if num_connections > 0:
        original_radii = actual_target_connections['Orbital Radius']
        min_rad, max_rad = original_radii.min(), original_radii.max()
        rad_range = max_rad - min_rad if max_rad > min_rad else 1.0
        min_visual_radius, max_visual_radius = 3.0, 10.0
        visual_range = max_visual_radius - min_visual_radius
        thetas = np.linspace(0, 2 * np.pi, num_connections, endpoint=False)

        for i, (index, row) in enumerate(actual_target_connections.iterrows()):
            scaled_radius = ((row['Orbital Radius'] - min_rad) / rad_range) * visual_range + min_visual_radius
            radii_for_rings.append(scaled_radius)
            theta = thetas[i]
            pos[row['target']] = (scaled_radius * np.cos(theta), scaled_radius * np.sin(theta), 0)

    # --- Add Orbital Rings ---
    for r in sorted(list(set(radii_for_rings))):
        theta_ring = np.linspace(0, 2 * np.pi, 100)
        fig.add_trace(go.Scatter3d(
            x=r * np.cos(theta_ring), y=r * np.sin(theta_ring), z=np.zeros(100),
            mode='lines',
            line=dict(color='rgba(255, 255, 255, 0.2)', width=1, dash='solid'),
            hoverinfo='none'
        ))

    scene_annotations = []

    # --- Loop Through Each Node to Draw Them ---
    for node_name, coords in pos.items():
        center_x, center_y, center_z = coords
        is_source = (node_name == source_ticker)

        screener_info_row = screener_data_df[screener_data_df['code'] == node_name]
        if screener_info_row.empty: continue
        screener_info = screener_info_row.iloc[0]

        market_cap = screener_info.get('market_capitalization', 0)
        market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"
        min_visual_size, max_visual_size = 0.6, 1.5

        if is_source:
            hover_text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                          f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                          f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                          f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                          f"Market Cap: {market_cap_str}")
            node_color = get_node_color(source_info['gravitational_impact'], -80, 80)
            radius = min_visual_size + (source_info['source_planet_radius'] * (max_visual_size - min_visual_size))
            subdivisions = 2
        else:
            processed_info = ticker_connections[ticker_connections['target'] == node_name].iloc[0]
            hover_text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                          f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                          f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                          f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                          f"Daily Change: {processed_info['Daily Change']:.2f}%<br>"
                          f"Market Cap: {market_cap_str}")
            node_color = get_node_color(processed_info['Daily Change'], -5, 5)
            radius = min_visual_size + (processed_info['Planet Radius'] * (max_visual_size - min_visual_size))
            subdivisions = 2

        vertices, faces, vertex_colors = create_low_poly_sphere(center_x, center_y, center_z, radius, node_color, subdivisions, texture_spots=15)
        fig.add_trace(go.Mesh3d(
            x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            vertexcolor=vertex_colors,
            opacity=1.0,
            flatshading=True,
            # Add lighting properties to make edges crisp
            lighting=dict(
                ambient=0.8,
                diffuse=0.5,
                specular=1,
                roughness=1
            ),
            hoverinfo='text',
            text=hover_text,
            hoverlabel=dict(bgcolor='#0f0524', font=dict(color='#EAEAEA', size=14), bordercolor='rgba(255, 255, 255, 0.3)')
        ))

        # Add labels as scene annotations
        scene_annotations.append(
            dict(
                x=center_x,
                y=center_y,
                z=center_z, # Position label in the center of the sphere
                text=f"<b>{node_name}</b>",
                showarrow=False,
                font=dict(color='white', size=14),
                bgcolor="rgba(0,0,0,0)",
                xanchor="center"
            )
        )


    # --- Configure Final Layout ---
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False),
            camera=dict(eye=dict(x=0.9 * zoom, y=0.9 * zoom, z=0.9 * zoom)),
            aspectmode='data',
            annotations=scene_annotations,
            bgcolor='rgba(0,0,0,0)'
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(family="'Space Grotesk', sans-serif", color='#EAEAEA', size=16)
    )
    return fig

# --- Dash App Definition & Styling ---
app = Dash(__name__, external_stylesheets=['https://fonts.googleapis.com/css2?family=Space+Grotesk:wght@400;700&display=swap'])
server = app.server

# --- New Theme Styles ---
THEME = {
    'background': '#0B041A',
    'text': '#EAEAEA',
    'primary': '#FFFFFF',
    'container_bg': 'rgba(30, 15, 60, 0.3)',
    'container_border': 'rgba(255, 255, 255, 0.1)'
}

# Starry background style
starry_background_style = {
    'backgroundColor': THEME['background'],
    'backgroundImage': 'radial-gradient(circle, white 0.5px, transparent 1.5px), radial-gradient(circle, white 1px, transparent 2px), radial-gradient(circle, white 0.5px, transparent 1.5px)',
    'backgroundSize': '350px 350px, 250px 250px, 150px 150px',
    'backgroundPosition': '0 0, 40px 60px, 130px 270px',
    'color': THEME['text'],
    'fontFamily': "'Space Grotesk', sans-serif",
    'minHeight': '100vh',
    'padding': '20px'
}

font_style = {'fontFamily': "'Space Grotesk', sans-serif", 'color': THEME['text']}
container_style = {
    'backgroundColor': THEME['container_bg'],
    'border': f"1px solid {THEME['container_border']}",
    'padding': '25px 30px',
    'borderRadius': '12px',
    'backdropFilter': 'blur(10px)',
    'width': '100%',
    'boxSizing': 'border-box'
}
header_style = {**font_style, 'color': THEME['text'], 'textAlign': 'center', 'fontWeight': 'bold', 'marginTop': 0, 'marginBottom': '20px', 'fontSize': '22px'}

# Assume dataframes are loaded
ticker_options = [{'label': row['name'] + f" ({row['code']})", 'value': row['code']} for index, row in screener_data_df.iterrows()]
default_ticker = 'AAPL'

# --- App Layout ---
app.layout = html.Div(style=starry_background_style, children=[
    dcc.Store(id='zoom-level-store', data=1.5), # Store for zoom level
    html.H1("THE FINANCIAL OBSERVATORY", style={**font_style, 'textAlign': 'center', 'padding': '20px 0', 'fontSize': '32px', 'fontWeight': 'bold', 'letterSpacing': '4px'}),

    html.Div([
        dcc.Dropdown(id='ticker-dropdown', options=ticker_options, value=default_ticker, clearable=False, style={'height': '40px', 'color': 'black'})
    ], style={'width': '90%', 'maxWidth': '500px', 'margin': '0 auto 20px auto', 'backgroundColor': THEME['container_bg'], 'border': f"1px solid {THEME['container_border']}", 'borderRadius': '12px', 'backdropFilter': 'blur(10px)', 'padding': '5px'}),

    html.P(id='prediction-summary-text', style={'textAlign': 'center', 'padding': '10px 0', 'fontSize': '18px', 'color': THEME['text']}),

    html.Div(id='graph-container', style={'height': '50vh', 'width': '98%', 'margin': 'auto', 'borderRadius': '15px', 'boxShadow': '0 0 25px 5px rgba(255, 255, 255, 0.15)'}),

    # Zoom slider container
    html.Div([
        dcc.Slider(
            id='zoom-slider',
            min=0.5,
            max=3,
            step=0.1,
            value=1.5,
            marks={
                0.5: {'label': 'Zoom In', 'style': {'color': 'white', 'fontWeight': 'bold'}},
                3: {'label': 'Zoom Out', 'style': {'color': 'white', 'fontWeight': 'bold'}}
            },
            className='themed-slider'
        )
    ], style={'width': '80%', 'maxWidth': '600px', 'margin': '30px auto 20px auto'}),

    html.Div(id='info-panels-container')
])


# --- Callbacks ---

# Combined callback for updating data and panels when ticker changes
@app.callback(
    [Output('info-panels-container', 'children'),
     Output('prediction-summary-text', 'children'),
     Output('graph-container', 'children')],
    [Input('ticker-dropdown', 'value')],
    [State('zoom-level-store', 'data')]
)
def update_content_on_ticker_change(selected_ticker, zoom_level):

    # Determine if it's the weekend
    today_weekday = datetime.today().weekday()
    is_weekend = today_weekday >= 5  # Saturday is 5, Sunday is 6

    global processed_data_df, source_data_df
    processed_data_df, source_data_df = process_and_score_stocks(
        six_month_spearman_lagged_correlations, three_month_spearman_lagged_correlations, screener_data_df,
        selected_ticker, min_nodes, max_nodes, threshold_percent
    )

    if processed_data_df.empty or source_data_df.empty:
        empty_fig = go.Figure().update_layout(title=f"Data not available for {selected_ticker}", title_x=0.5, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')
        empty_graph = dcc.Graph(id='network-graph', figure=empty_fig, style={'height': '100%'})
        no_data_msg = html.Div(f"Data not available for {selected_ticker}", style={'textAlign': 'center', 'padding': '20px'})
        return no_data_msg, "", empty_graph

    # Regenerate graph with the stored zoom level
    graph = dcc.Graph(id='network-graph', figure=solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df, zoom_level), style={'height': '100%'})

    # --- Generate data for panels and summary text ---
    star_info_screener = screener_data_df[screener_data_df['code'] == selected_ticker].iloc[0]
    star_info_source = source_data_df[source_data_df['ticker'] == selected_ticker].iloc[0]
    grav_impact = star_info_source.get('gravitational_impact', 0)
    net_gravitational_force = star_info_source.get('net_gravitational_force', 0)
    max_potential_force = star_info_source.get('max_potential_force', 0)

    planets_df = processed_data_df[processed_data_df['source'] == selected_ticker].copy()
    max_gravitational_force = planets_df['signed_gravitational_force'].abs().sum()

    # --- Conditional text based on day of the week ---
    prediction_day_text = "on Monday" if is_weekend else "today"
    daily_change_header = ["Friday's", html.Br(), "Daily Close"] if is_weekend else ["Yesterday's", html.Br(), "Daily Change"]
    predictions_header_text = "Monday's Top Predictions" if is_weekend else "Top Predictions"


    # Summary text
    source_name = star_info_screener.get('name', selected_ticker)
    direction = "increase" if grav_impact >= 0 else "decrease"
    strength = grav_impact
    prediction_summary = f"{source_name} ({selected_ticker}) is predicted to {direction} {prediction_day_text} with a prediction strength of {strength:.2f}%."

    # Star Info Panel
    star_color = get_node_color(grav_impact, -80, 80)
    star_image_src = create_model_image_svg(star_color, subdivisions=2, texture_spots_count=10)

    star_text_content = html.Div([
        html.P(
            f"{source_name} ({selected_ticker}) is expected to {direction} {prediction_day_text} with a prediction strength of {grav_impact:.2f}%. "
            f"The prediction strength is calculated based on how correlated stocks are with the next day performance of {source_name} ({selected_ticker})."
        ),
        html.P(
            f"If all of the planets in the solar system had perfect correlations, the maximum gravitational force that could exist in the system would be {max_potential_force:.2f}. "
            f"The net gravitational force acting on the star right now is {net_gravitational_force:.2f}.",
            style={'marginTop': '10px'}
        )
    ], style={
        'maxHeight': '120px',
        'overflowY': 'auto',
        'paddingRight': '15px',
        'maskImage': 'linear-gradient(to bottom, black 60%, transparent 100%)',
        'WebkitMaskImage': 'linear-gradient(to bottom, black 60%, transparent 100%)'
    })

    star_info_panel = html.Div([
        html.H3("Star Information", style=header_style),
        html.Div([
            html.Img(src=star_image_src, style={'height': '100px', 'width': '100px', 'marginRight': '20px', 'flexShrink': '0'}),
            star_text_content
        ], style={'display': 'flex', 'alignItems': 'center'})
    ], style=container_style)

    # --- Predictions Panel ---
    prediction_items = []
    if 'top_positive_impacts' in globals() and not top_positive_impacts.empty and 'top_negative_impacts' in globals() and not top_negative_impacts.empty:
        combined_impacts = pd.concat([top_positive_impacts.head(5), top_negative_impacts.head(5)])

        def create_prediction_item(row):
            ticker = row['ticker']
            screener_row = screener_data_df[screener_data_df['code'] == ticker]
            name = screener_row['name'].iloc[0] if not screener_row.empty else ticker
            display_text = f"{name} ({ticker})"
            return html.Div([
                html.Span(display_text),
                html.Span(f"{row['gravitational_impact']:.2f}%", style={'color': '#4ade80' if row['gravitational_impact'] > 0 else '#f87171', 'fontWeight': 'bold'})
            ],
            id={'type': 'prediction-item', 'index': ticker},
            n_clicks=0,
            style={'display': 'flex', 'justifyContent': 'space-between', 'padding': '8px 0', 'borderBottom': f'1px solid {THEME["container_border"]}', 'cursor': 'pointer'}
            )

        prediction_items = [create_prediction_item(row) for _, row in combined_impacts.iterrows()]

    else:
         prediction_items = [html.Div("Top predictions not available.")]

    predictions_panel = html.Div([html.H3(predictions_header_text, style=header_style), html.Div(prediction_items)], style=container_style)


    # --- Legend Panel ---
    legend_gradient_style = {'height': '15px', 'width': '100%', 'maxWidth': '300px', 'borderRadius': '5px', 'marginBottom': '8px', 'background': f"linear-gradient(to right, {RED_SPECTRUM['dark']}, {RED_SPECTRUM['light']} 49.9%, {GREEN_SPECTRUM['light']} 50.1%, {GREEN_SPECTRUM['dark']})"}
    legend_content = html.Div([
        html.Div(style=legend_gradient_style),
        html.Div([html.Span("Decrease"), html.Span("Increase")], style={'display': 'flex', 'justifyContent': 'space-between', 'fontSize': '12px', 'width': '100%', 'maxWidth': '300px'})
    ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'})
    legend_panel_container = html.Div([
        html.H3("Legend", style={**header_style, 'marginBottom': '15px', 'textAlign': 'center'}),
        legend_content
    ], style={**container_style, 'padding': '15px 20px', 'width': '100%', 'align-self': 'center'})

    # --- Planet Info Table ---
    headers = [
        "Code",
        "Name",
        ["Correlation", html.Br(), f"with {selected_ticker}"],
        "Market Cap",
        daily_change_header,
        "Grav. Force"
    ]
    table_header = [html.Thead(html.Tr([html.Th(col, style={'padding': '12px', 'textAlign': 'left', 'borderBottom': f"2px solid {THEME['container_border']}"}) for col in headers]))]

    table_rows = []
    if not planets_df.empty:
        for _, p_row in planets_df.iterrows():
            s_info = screener_data_df[screener_data_df['code'] == p_row['target']].iloc[0]
            planet_color = get_node_color(p_row['Daily Change'], -5, 5)
            planet_image_src = create_model_image_svg(planet_color, subdivisions=2, texture_spots_count=5)

            ticker_cell = html.Div([
                html.Img(src=planet_image_src, style={'height': '40px', 'width': '40px', 'marginRight': '10px'}),
                html.Span(p_row['target'])
            ], style={'display': 'flex', 'alignItems': 'center'})

            table_rows.append(html.Tr([
                html.Td(ticker_cell, style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(s_info['name'], style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['unified_correlation']:.2%}", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"${s_info['market_capitalization']/1e9:.2f}B", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['Daily Change']:.2f}%", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['signed_gravitational_force']:.2f}", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
            ]))

    # Dynamically set the height of the table container
    table_wrapper_style = {
        'overflowX': 'auto',
        'maskImage': 'linear-gradient(to right, black 95%, transparent 100%)',
        'WebkitMaskImage': 'linear-gradient(to right, black 95%, transparent 100%)'
    }
    if len(planets_df) > 5:
        table_wrapper_style['maxHeight'] = '300px'
        table_wrapper_style['overflowY'] = 'auto'

    planet_table_panel = html.Div([
        html.H3("Planet Information", style=header_style),
        html.Div(html.Table(table_header + [html.Tbody(table_rows)], style={'width': '100%', 'borderCollapse': 'collapse'}), style=table_wrapper_style)
    ], style=container_style)

    # Assemble the info panels layout
    info_panels = html.Div([
        legend_panel_container,
        star_info_panel,
        planet_table_panel,
        predictions_panel
    ], style={'display': 'flex', 'flexDirection': 'column', 'gap': '20px', 'padding': '20px 0'})

    return info_panels, prediction_summary, graph

# Callback to update zoom level in store
@app.callback(
    Output('zoom-level-store', 'data'),
    Input('zoom-slider', 'value')
)
def update_zoom_store(zoom_level):
    return zoom_level

# Callback to update graph only when zoom level changes
@app.callback(
    Output('graph-container', 'children', allow_duplicate=True),
    Input('zoom-level-store', 'data'),
    State('ticker-dropdown', 'value'),
    prevent_initial_call=True
)
def update_graph_on_zoom(zoom_level, selected_ticker):
    if selected_ticker is None:
        raise PreventUpdate

    # We need the dataframes, so we might have to re-run the processing if they are not available globally
    # For this example, we assume they exist from the last ticker change
    if 'processed_data_df' not in globals() or 'source_data_df' not in globals():
        raise PreventUpdate

    figure = solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df, zoom_level)
    return dcc.Graph(id='network-graph', figure=figure, style={'height': '100%'})

# Callback to update dropdown when a prediction is clicked
@app.callback(
    Output('ticker-dropdown', 'value'),
    Input({'type': 'prediction-item', 'index': ALL}, 'n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_from_prediction_click(n_clicks):
    if not any(n_clicks):
        raise PreventUpdate

    ctx = callback_context
    if not ctx.triggered:
        raise PreventUpdate

    triggered_id_str = ctx.triggered[0]['prop_id'].split('.')[0]

    if not triggered_id_str:
        raise PreventUpdate

    triggered_id = json.loads(triggered_id_str)

    new_ticker = triggered_id['index']

    return new_ticker

# --- Run the App ---
if __name__ == '__main__':
    app.run(debug=True, port=8051)
